In [ ]:
import pandas as pd
import os
import ee
import datetime
import pandas as pd

ee.Authenticate()
ee.Initialize()

# BREAK CSV INTO CHUNKS (ONLY IF NECESSARY)

In [14]:
# Input CSV file path
input_file = "Karnataka_Datasets/Karnataka_Dataset_Across_Regions.csv"

# Output directory for chunks
output_dir = "Karnataka_Datasets/Across/Chunks"
os.makedirs(output_dir, exist_ok=True)

# Set the chunk size
chunk_size = 5000

# Read the CSV in chunks
for i, chunk in enumerate(pd.read_csv(input_file, chunksize=chunk_size)):
    chunk_filename = os.path.join(output_dir, f"chunk_{i+1}.csv")
    chunk.to_csv(chunk_filename, index=False)
    print(f"Saved {chunk_filename}")


Saved Karnataka_Datasets/Across/Chunks\chunk_1.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_2.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_3.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_4.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_5.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_6.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_7.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_8.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_9.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_10.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_11.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_12.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_13.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_14.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_15.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_16.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_17.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_18.csv
Saved Karnataka_Datasets/Across/Chunks\chunk_19.csv
Saved Karnataka_Datas

In [8]:
# Load CSV
input_csv_path = 'Karnataka_Datasets/Across/Chunks/chunk_10.csv'
df = pd.read_csv(input_csv_path)

# Convert to ee.FeatureCollection
def row_to_feature(row):
    point = ee.Geometry.Point([row['Longitude'], row['Latitude']])
    buffer = point.buffer(10)
    return ee.Feature(buffer, {
        'Latitude': row['Latitude'],
        'Longitude': row['Longitude'],
        'Crop_Name': row['Crop_Name']
    })

features = [row_to_feature(row) for _, row in df.iterrows()]
feature_collection = ee.FeatureCollection(features)

# Dates
start_date = datetime.date(2021, 4, 1)
end_date = datetime.date(2022, 3, 31)
delta = datetime.timedelta(days=5)

dates = []
current = start_date
while current <= end_date:
    dates.append(current)
    current += delta

# Add indices function
def add_indices(image):
    # Divide the bands by 10000 to scale the reflectance values correctly
    image = image.multiply(0.0001)  # This is equivalent to dividing by 10000
    
    # Add NDVI and GCVI indices
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    gcvi = image.select('B8').divide(image.select('B3')).subtract(1).rename('GCVI')
    
    return image.addBands([ndvi, gcvi])



# List to hold feature collections for each date
all_fc = []

for date in dates:
    start = ee.Date(str(date))
    end = start.advance(5, 'day')

    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)) \
        .select(['B3', 'B4', 'B8', 'B8A', 'B11', 'B12'])  # Added SCL band

    # Apply SCL cloud mask

    img = s2.median()
    img = add_indices(img)

    band_names = img.bandNames().getInfo()
    if not band_names:
        print(f"⚠️ No image found for {date}, skipping.")
        continue

    reduced = img.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=10
    )

    # Tag each feature with the date
    reduced = reduced.map(lambda f: f.set('Date', str(date)))
    all_fc.append(reduced)

# Merge all results
merged_fc = ee.FeatureCollection(all_fc).flatten()

# Export to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=merged_fc,
    description='Karnataka_Across_Data',
    folder='EarthEngine_VI',
    fileNamePrefix='Karnataka_Chunk_10_90',
    fileFormat='CSV'
)
task.start()

print("✅ Export started. Monitor progress at: https://code.earthengine.google.com/tasks")


✅ Export started. Monitor progress at: https://code.earthengine.google.com/tasks


# EXTRACT S2 DATA FOR A SINGLE FILE

In [10]:
# Load CSV
input_csv_path = 'Karnataka_Datasets/Across/Chunks/chunk_10.csv'
df = pd.read_csv(input_csv_path)

# Convert to ee.FeatureCollection
def row_to_feature(row):
    point = ee.Geometry.Point([row['Longitude'], row['Latitude']])
    buffer = point.buffer(10)
    return ee.Feature(buffer, {
        'Latitude': row['Latitude'],
        'Longitude': row['Longitude'],
        'Crop_Name': row['Crop_Name']
    })

features = [row_to_feature(row) for _, row in df.iterrows()]
feature_collection = ee.FeatureCollection(features)

# Dates
start_date = datetime.date(2021, 4, 1)
end_date = datetime.date(2022, 3, 31)
delta = datetime.timedelta(days=5)

dates = []
current = start_date
while current <= end_date:
    dates.append(current)
    current += delta

# Add NDVI and GCVI indices
def add_indices(image):
    image = image.multiply(0.0001)  # scale reflectance
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    gcvi = image.select('B8').divide(image.select('B3')).subtract(1).rename('GCVI')
    return image.addBands([ndvi, gcvi])

# QA60 cloud masking function
def maskS2cloudsQA60(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10  # Cloud
    cirrus_bit_mask = 1 << 11  # Cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)

# List to hold feature collections for each date
all_fc = []

# Loop over all dates
for date in dates:
    start = ee.Date(str(date))
    end = start.advance(5, 'day')

    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
        .map(maskS2cloudsQA60) \
        .select(['B3', 'B4', 'B8', 'B8A', 'B11', 'B12', 'QA60'])  # Include QA60 for masking

    img = s2.median()
    img = add_indices(img)

    band_names = img.bandNames().getInfo()
    if not band_names:
        print(f"⚠️ No image found for {date}, skipping.")
        continue

    reduced = img.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=10
    )

    reduced = reduced.map(lambda f: f.set('Date', str(date)))
    all_fc.append(reduced)

# Merge all results
merged_fc = ee.FeatureCollection(all_fc).flatten()

# Export to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=merged_fc,
    description='Karnataka_Across_Data',
    folder='EarthEngine_VI_QA_90',
    fileNamePrefix='Karnataka_Chunk_10_QA',
    fileFormat='CSV'
)
task.start()

print("✅ Export started. Monitor progress at: https://code.earthengine.google.com/tasks")


✅ Export started. Monitor progress at: https://code.earthengine.google.com/tasks


# EXTRACT S2 DATA FOR A DIRECTORY

In [1]:
# Input directory containing all chunks
input_dir = 'Karnataka_Datasets/Across/Chunks_Rem/'

# Output settings
export_folder = 'EarthEngine_VI_QA_CloudProb'
export_description_prefix = 'Karnataka_Chunk'

# Define dates
start_date = datetime.date(2021, 4, 1)
end_date = datetime.date(2022, 3, 31)
delta = datetime.timedelta(days=5)

dates = []
current = start_date
while current <= end_date:
    dates.append(current)
    current += delta

# NDVI & GCVI function
def add_indices(image):
    image = ee.Image(image).multiply(0.0001)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    gcvi = image.select('B8').divide(image.select('B3')).subtract(1).rename('GCVI')
    return image.addBands([ndvi, gcvi])

# Cloud masking
def mask_clouds(image):
    qa60 = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    qa_mask = qa60.bitwiseAnd(cloud_bit_mask).eq(0).And(
              qa60.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    cloud_mask_img = ee.Image(image.get('cloud_mask'))
    
    return ee.Algorithms.If(
        cloud_mask_img,
        image.updateMask(qa_mask.And(cloud_mask_img.select('probability').lt(50))),
        image.updateMask(qa_mask)
    )

# Process each file in the input directory
for filename in os.listdir(input_dir):
    if not filename.endswith('.csv'):
        continue

    filepath = os.path.join(input_dir, filename)
    df = pd.read_csv(filepath)

    # Convert to FeatureCollection
    def row_to_feature(row):
        point = ee.Geometry.Point([row['Longitude'], row['Latitude']])
        return ee.Feature(point, {
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude'],
            'Crop_Name': row['Crop_Name']
        })

    features = [row_to_feature(row) for _, row in df.iterrows()]
    feature_collection = ee.FeatureCollection(features)

    all_fc = []

    for date in dates:
        start = ee.Date(str(date))
        end = start.advance(5, 'day')

        s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start, end) \
            .filterBounds(feature_collection) \
            .select(['B11', 'B12', 'B3', 'B4', 'B8', 'B8A', 'QA60'])

        s2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
            .filterDate(start, end) \
            .filterBounds(feature_collection)

        joined = ee.Join.saveFirst('cloud_mask').apply(
            primary=s2_sr,
            secondary=s2_clouds,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )

        def process_image(img):
            img = ee.Image(img)
            masked = ee.Image(mask_clouds(img))
            return add_indices(masked)

        processed = ee.ImageCollection(joined).map(process_image)

        # Check if any image is available
        if processed.size().lte(0).getInfo():
            print(f"⚠️ No image found for {date}, skipping.")
            continue

        img = processed.median()

        reduced = img.reduceRegions(
            collection=feature_collection,
            reducer=ee.Reducer.mean(),
            scale=10
        )

        reduced = reduced.map(lambda f: f.set('Date', str(date)))
        all_fc.append(reduced)

    if all_fc:
        merged_fc = ee.FeatureCollection(all_fc).flatten()

        chunk_name = os.path.splitext(filename)[0]
        task = ee.batch.Export.table.toDrive(
            collection=merged_fc,
            description=f'{export_description_prefix}_{chunk_name}_QA_CloudProb',
            folder=export_folder,
            fileNamePrefix=f'{chunk_name}_QA_CloudProb',
            fileFormat='CSV'
        )
        task.start()
        print(f"✅ Export started for {chunk_name}")
    else:
        print(f"⚠️ No valid images found for any dates in {filename}, skipping export.")


⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_23
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_24
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_25
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_26
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_27
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_28
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_3
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image found for 2021-12-12, skipping.
✅ Export started for chunk_4
⚠️ No image found for 2021-11-07, skipping.
⚠️ No image fo